# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

## 🎯 단계별 접근 가이드

### 1단계: 기본 구조 설정 (필수)
- [ ] Imports (ollama, gradio)
- [ ] 모델 설정 및 Ollama 확인
- [ ] System prompt 작성 (기술 튜터 역할)

### 2단계: 기본 Chat 함수 (필수)
- [ ] Gradio ChatInterface로 간단한 채팅 구현
- [ ] History 관리
- [ ] System prompt 적용

### 3단계: 스트리밍 추가 (필수)
- [ ] `ollama.chat(..., stream=True)` 사용
- [ ] Generator 함수로 `yield` 사용
- [ ] 실시간 응답 표시 확인

### 4단계: 모델 전환 기능 (필수)
- [ ] `gr.Dropdown`으로 모델 선택 UI 추가
- [ ] `additional_inputs`에 드롭다운 연결
- [ ] Chat 함수에서 모델 선택 반영

### 5단계: Tool 추가 (보너스)
- [ ] Tool 함수 정의 (예: 코드 검색, 문서 검색)
- [ ] Tool schema 정의
- [ ] `handle_tool_calls` 함수 구현
- [ ] Tool 호출 루프 처리

### 6단계: 고급 기능 (선택)
- [ ] 오디오 입력/출력
- [ ] 예시 질문 추가
- [ ] UI 커스터마이징

## 💡 구현 팁

1. **작은 것부터 시작**: 먼저 기본 채팅만 구현하고, 하나씩 기능 추가
2. **Week 1 Exercise 참고**: 이미 만든 코드를 재사용하세요
3. **Week 2 Day 3-5 참고**: Gradio, 스트리밍, Tool 예제가 있습니다
4. **에러 처리**: try-except로 Ollama 연결 확인
5. **테스트**: 각 단계마다 테스트하며 진행

## 📚 참고할 파일들

- `week1/week1 EXERCISE.ipynb` - 기본 구조 참고
- `week2/day3.ipynb` - Gradio ChatInterface 예제
- `week2/day4.ipynb` - Tool calling 예제
- `week2/day5.ipynb` - 완전한 예제

## Example Solution

아래는 완전한 예시 솔루션입니다. 먼저 직접 구현해보고, 막히면 참고하세요!

## 🚀 시작하기 전 체크리스트

아래 셀들을 순서대로 구현해보세요. 각 셀은 독립적으로 실행 가능하도록 만들어야 합니다.

**핵심 요구사항:**
1. ✅ Gradio UI (ChatInterface 사용)
2. ✅ 스트리밍 응답
3. ✅ System prompt로 전문성 추가
4. ✅ 모델 전환 기능
5. 🎁 보너스: Tool 사용

**시작 전 확인:**
- [ ] Ollama가 실행 중인가요? (`ollama list`로 확인)
- [ ] 최소 하나의 모델이 설치되어 있나요? (`ollama pull llama3.2`)

In [1]:
import ollama
import json
import gradio as gr

In [2]:
MODEL = "llama3.2"

try:
    ollama.list()
    print("✓ Ollama is available")
except Exception as e:
    print(f"⚠️ Ollama may not be running: {e}")


✓ Ollama is available


In [11]:
# System Prompt - 전문성 추가

system_prompt = """
You are a helpful technical tutor who answers questions about Python code, 
software engineering, machine learning, and LLMs.

You are a senior software engineer with 20 years of experience in the field.
You are also an expert in teaching and explaining complex ideas.

When answering questions:
- Provide precise and detailed explanations
- Use examples when helpful
- Break down complex concepts into understandable parts
- If you don't know something, say so honestly

Always be accurate and helpful.
"""

### Add Tools

In [3]:
# Tools
def search_code_examples(topic: str) -> str:
    examples_db = {
        "generator": "def my_generator():\n    yield 1\n    yield 2\n    yield 3",
        "decorator": "@my_decorator\ndef my_function():\n    pass",
        "context manager": "with open('file.txt') as f:\n    content = f.read()",
        "list comprehension": "[x**2 for x in range(10) if x % 2 == 0]",
        "async": "async def fetch_data():\n    async with aiohttp.ClientSession() as session:\n        async with session.get(url) as response:\n            return await response.json()"
    }
    topic_lower = topic.lower()
    for key, example in examples_db.items():
        if key in topic_lower:
            return f"Code example for '{key}':\n```python\n{example}\n```"
    
    return f"No code example found for '{topic}'. Available topics: {', '.join(examples_db.keys())}"

# Tool definition
code_search_function = {
    "name": "search_code_examples",
    "description": "Search for Python code examples by topic (e.g., generator, decorator, async, list comprehension)",
    "parameters": {
        "type": "object",
        "properties": {
            "topic": {
                "type": "string",
                "description": "The programming topic or concept to search for code examples"
            }            
        },
        "required": ["topic"],
        "additionalProperties": False
    }
}

In [4]:
def get_python_docs(concept: str) -> str:
    """Python 개념에 대한 문서를 검색하는 도구"""
    docs_db = {
        "yield": "The 'yield' keyword is used to create a generator function. It pauses function execution and returns a value to the caller, but retains enough state to enable the function to resume where it left off.",
        "decorator": "A decorator is a design pattern in Python that allows a user to add new functionality to an existing object without modifying its structure.",
        "async": "Async/await syntax allows you to write asynchronous code that looks like synchronous code, making it easier to handle concurrent operations."
    }
    
    concept_lower = concept.lower()
    for key, doc in docs_db.items():
        if key in concept_lower:
            return f"Documentation for '{key}':\n{doc}"
    
    return f"No documentation found for '{concept}'. Available concepts: {', '.join(docs_db.keys())}"

docs_search_function = {
    "name": "get_python_docs",
    "description": "Get documentation or explanation for a Python concept (e.g., yield, decorator, async)",
    "parameters": {
        "type": "object",
        "properties": {
            "concept": {
                "type": "string",
                "description": "The Python concept to get documentation for"
            }
        },
        "required": ["concept"],
        "additionalProperties": False
    }
}

In [5]:
tools = [
    {"type": "function", "function": code_search_function},
    {"type": "function", "function": docs_search_function}
]

print("Tools defined:")
print("- search_code_examples: Search for Python code examples")
print("- get_python_docs: Get Python documentation")

Tools defined:
- search_code_examples: Search for Python code examples
- get_python_docs: Get Python documentation


In [ ]:
def handle_tool_calls(message):
    responses = []

    if not message.get("tool_calls"):
        return responses

    for tool_call in message["tool_calls"]:
        tool_name = tool_call["function"]["name"]
        arguments = tool_call["function"].get("arguments", {})

        if isinstance(arguments, str):
            try:
                arguments = json.loads(arguments)
            except json.JSONDecodeError:
                arguments = {}
        
        elif not isinstance(arguments, dict):
            arguments = {}

        # Tool call ID 추출 (선택적)
        tool_call_id = tool_call.get('id')

        if tool_name == "search_code_examples":
            topic = arguments.get("topic", "")
            result = search_code_examples(topic)

        elif tool_name == "get_python_docs":
            concept = arguments.get("concept", "")
            result = get_python_docs(concept)

        else:
            result = f"Unknown tool: {tool_name}"

        response = {
            "role": "tool",
            "content": result # Tool 실행 결과
        }

        if tool_call_id:
            response["tool_call_id"] = tool_call_id

        responses.append(response)

    return responses

In [ ]:
# # Tool 호출 처리 함수
# def handle_tool_calls(message):
#     """
#     LLM이 호출한 Tool들을 실행하고 결과를 반환하는 함수
    
#     이 함수는 LLM의 응답 메시지에서 tool_calls를 추출하여,
#     각 tool을 실행하고 그 결과를 LLM이 이해할 수 있는 형식으로 반환합니다.
    
#     Args:
#         message (dict): LLM의 응답 메시지 딕셔너리
#                        예: {
#                            "role": "assistant",
#                            "content": "...",
#                            "tool_calls": [
#                                {
#                                    "function": {"name": "get_python_docs", "arguments": {...}},
#                                    "id": "call_123"
#                                }
#                            ]
#                        }
    
#     Returns:
#         list: Tool 실행 결과를 담은 메시지 리스트
#               각 요소는 {"role": "tool", "content": "...", "tool_call_id": "..."} 형식
    
#     동작 흐름:
#     1. message에서 tool_calls가 있는지 확인
#     2. 각 tool_call을 순회하며:
#        - Tool 이름 추출
#        - Arguments 파싱 (문자열이면 JSON 파싱)
#        - 해당 Tool 함수 실행
#        - 결과를 tool 메시지 형식으로 변환
#     3. 모든 결과를 리스트로 반환
#     """
#     # 📦 Tool 실행 결과를 담을 리스트 초기화
#     # 각 tool의 실행 결과가 이 리스트에 추가됩니다
#     responses = []
    
#     # 🔍 Tool 호출이 있는지 확인
#     # message.get('tool_calls')는 tool_calls 키가 없으면 None 반환
#     # tool_calls가 없으면 빈 리스트 반환 (빠른 종료)
#     if not message.get('tool_calls'):
#         return responses
    
#     # 🔄 각 tool_call을 순회하며 처리
#     # LLM이 여러 tool을 동시에 호출할 수 있으므로 반복문 사용
#     for tool_call in message['tool_calls']:
#         # 📛 Tool 이름 추출
#         # tool_call['function']['name']에서 호출하려는 함수 이름 가져오기
#         # 예: "get_python_docs", "search_code_examples"
#         tool_name = tool_call['function']['name']
        
#         # 🔧 Arguments 파싱 (매우 중요!)
#         # LLM이 전달한 인자들을 안전하게 파싱합니다
#         # Ollama는 arguments를 문자열 또는 딕셔너리로 전달할 수 있습니다
        
#         # 기본값으로 빈 딕셔너리 설정
#         arguments = tool_call['function'].get('arguments', {})
        
#         # Case 1: arguments가 문자열인 경우 (JSON 문자열)
#         # 예: '{"concept": "yield"}' -> {"concept": "yield"}
#         if isinstance(arguments, str):
#             try:
#                 # JSON 문자열을 딕셔너리로 파싱
#                 arguments = json.loads(arguments)
#             except json.JSONDecodeError:
#                 # JSON 파싱 실패 시 빈 딕셔너리로 설정 (에러 방지)
#                 arguments = {}
        
#         # Case 2: arguments가 딕셔너리가 아닌 경우
#         # 예상치 못한 타입이면 빈 딕셔너리로 설정
#         elif not isinstance(arguments, dict):
#             arguments = {}
        
#         # 🆔 Tool call ID 추출 (선택적)
#         # 각 tool 호출을 추적하기 위한 고유 ID
#         # 일부 모델은 이 ID를 제공하지 않을 수 있음
#         tool_call_id = tool_call.get('id')
        
#         # 🛠️ Tool 실행
#         # tool_name에 따라 적절한 함수를 호출합니다
#         if tool_name == "search_code_examples":
#             # 코드 예시 검색 tool 호출
#             topic = arguments.get('topic', '')  # 'topic' 키에서 값 추출, 없으면 빈 문자열
#             result = search_code_examples(topic)  # 실제 함수 실행
        
#         elif tool_name == "get_python_docs":
#             # Python 문서 검색 tool 호출
#             concept = arguments.get('concept', '')  # 'concept' 키에서 값 추출, 없으면 빈 문자열
#             result = get_python_docs(concept)  # 실제 함수 실행
        
#         else:
#             # 알 수 없는 tool인 경우 에러 메시지 반환
#             result = f"Unknown tool: {tool_name}"
        
#         # 📤 Tool 실행 결과를 LLM이 이해할 수 있는 메시지 형식으로 변환
#         # LLM은 "role": "tool"인 메시지를 받아야 합니다
#         response = {
#             "role": "tool",  # Tool 응답임을 나타내는 역할
#             "content": result  # Tool 실행 결과
#         }
        
#         # Tool call ID가 있으면 추가
#         # LLM이 여러 tool을 호출했을 때 어떤 tool의 결과인지 구분하기 위함
#         if tool_call_id:
#             response["tool_call_id"] = tool_call_id
        
#         # 📋 결과를 리스트에 추가
#         # 여러 tool이 호출되었을 경우 모든 결과를 수집
#         responses.append(response)
    
#     # ✅ 모든 tool 실행 결과 반환
#     # 이 리스트는 chat 함수에서 messages에 추가되어 LLM에게 전달됩니다
#     return responses

In [ ]:
def chat(message, history, model_choice=None):
    # 모델 선택 (model_choice가 없으면 기본 모델 사용)
    model = model_choice if model_choice else MODEL
    
    # history를 메시지 형식으로 변환    
    messages = [{"role": "system", "content": system_prompt}]
    # 이전 대화들 추가
    for h in history:
        messages.append({"role": h["role"], "content": h["content"]})
    # 현재 사용자 메시지 추가
    messages.append({"role": "user", "content": message})

    while True: 
        stream = ollama.chat(model=model, messages=messages, tools=tools, stream=True)

        # streamling 응답 수집
        response_text = ""
        assistant_message = None

        # stream
        for chunk in stream:
            if chunk.get('message'):
                msg = chunk['message']
                if msg.get('content'):
                    response_text += msg['content']
                    yield response_text
                
                # tool calls가 있으면 메시지 저장
                if msg.get('tool_calls') and not assistant_message:
                    assistant_message = msg.copy()
                    if response_text:
                        assistant_message['content'] = response_text

        if not assistant_message or not assistant_message.get('tool_calls'):
            break
        
        # tool calls 처리
        tool_responses = handle_tool_calls(assistant_message)

        # assistant 메시지와 tool 응답을 메시지에 추가
        messages.append(assistant_message)
        messages.extend(tool_responses)
        
        

In [ ]:
# # 메인 Chat 함수 - 스트리밍, 모델 전환, Tool 지원

# def chat(message, history, model_choice):
#     """
#     메인 채팅 함수
#     - 스트리밍 지원
#     - 모델 전환 지원
#     - Tool 호출 지원
#     """
#     # 모델 선택
#     model = AVAILABLE_MODELS.get(model_choice, DEFAULT_MODEL)
    
#     # History를 메시지 형식으로 변환
#     messages = [{"role": "system", "content": system_prompt}]
    
#     # 이전 대화 추가
#     for h in history:
#         messages.append({"role": h["role"], "content": h["content"]})
    
#     # 현재 사용자 메시지 추가
#     messages.append({"role": "user", "content": message})
    
#     # Tool 호출 처리 루프
#     while True:
#         # 응답 가져오기 (스트리밍으로)
#         stream = ollama.chat(model=model, messages=messages, tools=tools, stream=True)
        
#         # 스트리밍 응답 수집
#         response_text = ""
#         assistant_message = None
        
#         for chunk in stream:
#             if chunk.get('message'):
#                 msg = chunk['message']
#                 if msg.get('content'):
#                     response_text += msg['content']
#                     yield response_text
                
#                 # Tool calls가 있으면 메시지 저장
#                 if msg.get('tool_calls') and not assistant_message:
#                     assistant_message = msg.copy()
#                     if response_text:
#                         assistant_message['content'] = response_text
        
#         # Tool 호출이 없으면 종료
#         if not assistant_message or not assistant_message.get('tool_calls'):
#             break
        
#         # Tool 호출 처리
#         tool_responses = handle_tool_calls(assistant_message)
        
#         # Assistant 메시지와 tool 응답을 메시지에 추가
#         messages.append(assistant_message)
#         messages.extend(tool_responses)

In [15]:
# Gradio UI 생성

# 모델 선택 드롭다운 (모델 전환 기능 사용 시)
model_dropdown = gr.Dropdown(
    choices=["llama3.2", "llama3.1", "qwen2.5", "mistral"],
    value="llama3.2",
    label="Select Model",
    info="Choose which Ollama model to use"
)

# ChatInterface 생성 (스트리밍 지원)
chat_interface = gr.ChatInterface(
    fn=chat,
    type="messages",
    title="🤖 Technical Q&A Assistant",
    description="""
    Ask technical questions about Python, software engineering, ML, and LLMs!
    
    Features:
    - ✨ Streaming responses
    - 🔄 Model switching
    - 🛠️ Tool support (code examples, documentation)
    - 💡 Expert explanations
    
    Try asking:
    - "Explain generators in Python"
    - "What is a decorator?"
    - "Show me an async example"
    """,
    additional_inputs=[model_dropdown],  # 모델 선택 드롭다운 추가
    examples=[
        # additional_inputs를 사용할 때는 각 예시가 [message, model_choice] 형식이어야 합니다
        ["Explain what this code does: yield from {x for x in range(10)}", "llama3.2"],
        ["What is a Python decorator?", "llama3.2"],
        ["Show me an example of async/await", "llama3.2"],
        ["Explain list comprehensions", "llama3.2"],
    ]
)

chat_interface.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7888
* To create a public link, set `share=True` in `launch()`.
